In [1]:
print("ok")

ok


In [2]:
# Install some required packages

!pip install pypdf2
!pip install google-cloud-aiplatform
!pip install google-cloud-storage

  Using cached docstring_parser-0.16-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/5.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.3 MB ? eta -:--:--
   - -------------------------------------- 0.3/5.3 MB ? eta -:--:--
   --- ------------------------------------ 0.5/5.3 MB 1.1 MB/s eta 0:00:05
   ----- ---------------------------------- 0.8/5.3 MB 1.2 MB/s eta 0:00:04
   ------- -------------------------------- 1.0/5.3 MB 1.2 MB/s eta 0:00:04
   --------- ------------------------------ 1.3/5.3 MB 1.1 MB/s eta 0:00:04
   ----------- ---------------------------- 1.6/5.3 MB 1.1 MB/s eta 0:00:04
   ----------- ---------------------------- 1.6/5.3 MB 1.1 MB/s eta 0:00:04
   ----------- ---------------------------- 1.6/5.3 MB 1.1 MB/s eta 0:00:04
   ----------- ---------------------------- 1.6/5.3 MB 1.1 MB/s eta 0:00:04
   ----------- ---------------------------- 1.6/5.3 MB 1.1 MB/s eta 0:00:04
   ------------- --------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.10.14 requires protobuf<5,>=4.25.3, but you have protobuf 5.28.3 which is incompatible.
opentelemetry-proto 1.25.0 requires protobuf<5.0,>=3.19, but you have protobuf 5.28.3 which is incompatible.
streamlit 1.35.0 requires protobuf<5,>=3.20, but you have protobuf 5.28.3 which is incompatible.


In [3]:
from google.cloud import storage
from vertexai.language_models import TextEmbeddingModel
from google.cloud import aiplatform
import PyPDF2

import re
import os
import random
import json
import uuid


In [4]:
%ls

 Volume in drive C is OS
 Volume Serial Number is CE6C-D39A

 Directory of c:\Users\deepr\OneDrive\Desktop\python 1\Generative-AI-Mastery-Resources\LLMOps

26-10-2024  19:01    <DIR>          .
26-10-2024  19:01    <DIR>          ..
26-10-2024  19:01         1,164,471 bedrock demo.zip
26-10-2024  19:01         1,200,925 GenerativeAI-Project-RAG-Bedrock.zip
26-10-2024  19:01             3,119 LLM-Powered-Chatbot.zip
26-10-2024  19:25            52,643 RAG on VertexAI.ipynb
26-10-2024  19:01         2,028,599 vertexai demo.ipynb
26-10-2024  19:01            71,101 vertexai_llm_fine_tuning_supervised.ipynb
26-10-2024  19:01             1,190 Vertexai_local_demo.zip
               7 File(s)      4,522,048 bytes
               2 Dir(s)  62,977,605,632 bytes free


In [5]:
# Initialize some variable

# project="your_GCP_project_id"
location="us-central1"

pdf_path="stats.pdf"
bucket_name = "stats-content2024"
embed_file_path = "stats_embeddings.json"
sentence_file_path = "stats_sentences.json"
index_name="stats_index"


In [6]:
# helper

def extract_sentences_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            if page.extract_text() is not None:
                text += page.extract_text() + " "
    sentences = [sentence.strip() for sentence in text.split('. ') if sentence.strip()]
    return sentences

In [7]:
def generate_text_embeddings(sentences) -> list:
  # aiplatform.init(project=project,location=location)
  model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")
  embeddings = model.get_embeddings(sentences)
  vectors = [embedding.values for embedding in embeddings]
  return vectors

In [8]:
def generate_and_save_embeddings(pdf_path, sentence_file_path, embed_file_path):
    def clean_text(text):
        cleaned_text = re.sub(r'\u2022', '', text)  # Remove bullet points
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()  # Remove extra whitespaces and strip
        return cleaned_text

    sentences = extract_sentences_from_pdf(pdf_path)
    if sentences:
        embeddings = generate_text_embeddings(sentences)

        with open(embed_file_path, 'w') as embed_file, open(sentence_file_path, 'w') as sentence_file:
            for sentence, embedding in zip(sentences, embeddings):
                cleaned_sentence = clean_text(sentence)
                id = str(uuid.uuid4())

                embed_item = {"id": id, "embedding": embedding}
                sentence_item = {"id": id, "sentence": cleaned_sentence}

                json.dump(sentence_item, sentence_file)
                sentence_file.write('\n')
                json.dump(embed_item, embed_file)
                embed_file.write('\n')

In [9]:
def upload_file(bucket_name,file_path):
    storage_client = storage.Client()
    bucket = storage_client.create_bucket(bucket_name,location=location)
    blob = bucket.blob(file_path)
    blob.upload_from_filename(file_path)

In [10]:
def create_vector_index(bucket_name, index_name):
    lakeside_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name = index_name,
    contents_delta_uri = "gs://"+bucket_name,
    dimensions = 768,
    approximate_neighbors_count = 10,
    )

    lakeside_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name = index_name,
    public_endpoint_enabled = True
    )

    lakeside_index_endpoint.deploy_index(
    index = lakeside_index, deployed_index_id = index_name
    )

In [11]:
generate_and_save_embeddings(pdf_path,sentence_file_path,embed_file_path)
upload_file(bucket_name,sentence_file_path)

FileNotFoundError: [Errno 2] No such file or directory: 'stats.pdf'

In [19]:
create_vector_index(bucket_name, index_name)

INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Creating MatchingEngineIndex
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Create MatchingEngineIndex backing LRO: projects/509917135302/locations/us-central1/indexes/5993635795099975680/operations/6412509215151095808
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:MatchingEngineIndex created. Resource name: projects/509917135302/locations/us-central1/indexes/5993635795099975680
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:To use this MatchingEngineIndex in another session:
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:index = aiplatform.MatchingEngineIndex('projects/509917135302/locations/us-central1/indexes/5993635795099975680')
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Creating MatchingEngineIndexEndpoint
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Create MatchingEngineI

In [20]:
from vertexai.language_models import TextEmbeddingModel
from google.cloud import aiplatform
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
import json
import os

In [21]:
# project=”YOUR_GCP_PROJECT”
location="us-central1"
sentence_file_path = "stats_sentences.json"
index_name="stats_index" #Get this from the console or the previous step

In [25]:
# aiplatform.init(project=project,location=location)
# vertexai.init()
model = GenerativeModel("gemini-pro")
lakeside_index_ep = aiplatform.MatchingEngineIndexEndpoint(index_endpoint_name="1376179539650019328")

In [26]:
def generate_text_embeddings(sentences) -> list:
    model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")
    embeddings = model.get_embeddings(sentences)
    vectors = [embedding.values for embedding in embeddings]
    return vectors


def generate_context(ids,data):
    concatenated_names = ''
    for id in ids:
        for entry in data:
            if entry['id'] == id:
                concatenated_names += entry['sentence'] + "\n"
    return concatenated_names.strip()



def load_file(sentence_file_path):
  data = []
  with open(sentence_file_path,'r') as f:
    for line in f:
      entry = json.loads(line)
      data.append(entry)
  return data





In [27]:
data=load_file(sentence_file_path)
data

[{'id': 'd587cca8-5563-4321-9a6f-1128ab760880',
  'sentence': 'Importance and the use of correlation in Statistics Introduction Correlation is a statistical measure that expresses the extent to which two variables are linearly related. It is a common tool for describing simple relationships without making a statement about cause and effect. Correlation coefficients range from -1 to 1, with a value of 0 indicating no linear relationship between the two variables, a value of 1 indicating a perfect positive linear relationship, and a value of -1 indicating a perfect negative linear relationship. Correlation is important in statistics because it can be used to 1. Identify relationships between variables: Correlation can be used to identify whether there is a relationship between two variables, and if so, whether the relationship is positive or negative. This information can be useful for understanding the relationships between different factors in a complex system. 2. Make predictions: If 

In [38]:
query=["what is correlation?"]

qry_emb=generate_text_embeddings(query)

In [35]:
# qry_emb

In [39]:
response = lakeside_index_ep.find_neighbors(
    deployed_index_id = index_name,
    queries = [qry_emb[0]],
    num_neighbors = 10
)

In [40]:
matching_ids = [neighbor.id for sublist in response for neighbor in sublist]

context = generate_context(matching_ids,data)
prompt=f"Based on the context delimited in backticks, answer the query. ```{context}``` {query}"

chat = model.start_chat(history=[])
response = chat.send_message(prompt)
print(response.text)

Correlation is a statistical measure that indicates the extent to which two variables are related. It can range from -1 to 1, where:

* **-1**: Perfect negative correlation. As one variable increases, the other variable decreases proportionally.
* **0**: No correlation. There is no relationship between the two variables.
* **+1**: Perfect positive correlation. As one variable increases, the other variable also increases proportionally.

Correlation does not imply causation, meaning that just because two variables are correlated does not mean that one causes the other. Other factors may be responsible for the relationship between the variables.

Correlation is commonly used in research to identify relationships between variables and to make predictions. However, it is important to note that correlation is not always a reliable indicator of causation.
